In [1]:
from imutils.perspective import four_point_transform
from imutils import contours 
import numpy as np
import argparse
import imutils
import cv2
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
def detect(img):

    answerKey = {0: 1,
                1:4,
                2:0,
                3:3,
                4:5}

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    edged = cv2.Canny(blurred, 75, 200)
    try:
        cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        docCnt = None

        if len(cnts) > 0:
            cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
            
            for c in cnts:
                print(c)
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, 0.02*peri, True)
                
                if len(approx) == 4:
                    docCnt = approx
                    break
                    
        paper = four_point_transform(img, docCnt.reshape(4,2))
        warped = four_point_transform(gray, docCnt.reshape(4,2))

        thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)

        questionCnts = []

        for c in cnts:
            (x, y, w, h) = cv2.boundingRect(c)
            ar = w / float(h)
            
            if w>= 20 and h >= 20 and ar >= 0.9 and ar<= 1.1:
                questionCnts.append(c)
                
        questionCnts = contours.sort_contours(questionCnts, method='top-to-bottom')[0]
        correct = 0

        for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
            cnts = contours.sort_contours(questionCnts[i:i +5])[0]
            bubbled = None
            for (j, c) in enumerate(cnts):
                mask = np.zeros(thresh.shape, dtype='uint8')
                cv2.drawContours(mask, [c], -1, 255, -1)

                mask = cv2.bitwise_and(thresh, thresh, mask=mask)
                total = cv2.countNonZero(mask)

                if bubbled is None or total > bubbled[0]:
                    bubbled = (total, j)
                    
            color = (255, 0, 0)
            k = answerKey[q]
            # check to see if the bubbled answer is correct
            if k == bubbled[1]:
                color = (0, 255, 0)
                correct += 1
            # draw the outline of the correct answer on the test
            cv2.drawContours(paper, [cnts[k]], -1, color, 3)
    except:
        correct = 0
        print('Couldnt find the test paper!')
        pass

    score = (correct / 5.0) * 100
    print("[INFO] score: {:.2f}%".format(score))
    cv2.putText(paper, "{:.2f}%".format(score), (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    return paper



In [5]:
video = cv2.VideoCapture(0)

while(True):
    _, frame = video.read()
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xff == 27:
        result = detect(frame)
        cv2.destroyAllWindows()
        video.release()
        break


while(True):
    cv2.imshow('result', result)
    if cv2.waitKey(1) & 0xff == 27:
        cv2.destroyAllWindows()
        break
        

[[[607  18]]

 [[607  22]]

 [[605  24]]

 [[605  25]]

 [[602  28]]

 [[602  29]]

 [[601  30]]

 [[601  31]]

 [[600  32]]

 [[600  33]]

 [[599  34]]

 [[599  37]]

 [[598  38]]

 [[598  41]]

 [[597  42]]

 [[597  43]]

 [[596  44]]

 [[594  42]]

 [[592  42]]

 [[591  41]]

 [[590  42]]

 [[588  42]]

 [[584  46]]

 [[584  47]]

 [[583  48]]

 [[583  49]]

 [[582  50]]

 [[582  52]]

 [[581  53]]

 [[581  54]]

 [[580  55]]

 [[580  56]]

 [[579  57]]

 [[579  58]]

 [[578  59]]

 [[578  60]]

 [[579  61]]

 [[578  62]]

 [[577  62]]

 [[579  64]]

 [[577  66]]

 [[575  64]]

 [[574  64]]

 [[573  63]]

 [[572  63]]

 [[571  64]]

 [[571  65]]

 [[568  68]]

 [[568  69]]

 [[566  71]]

 [[566  74]]

 [[565  75]]

 [[565  76]]

 [[564  77]]

 [[564  78]]

 [[563  79]]

 [[563  80]]

 [[562  81]]

 [[562  82]]

 [[561  83]]

 [[561  84]]

 [[560  85]]

 [[560  86]]

 [[559  87]]

 [[559  88]]

 [[558  89]]

 [[558  90]]

 [[553  95]]

 [[551  95]]

 [[549  93]]

 [[549  87]]

 [[550

In [21]:
video.release()